In [2]:
import bibtexparser as bp
import pandas as pd
import numpy as np
import yaml as yml

# CARREGA O ARQUIVO DE CONFIGURAÇÃO
dirConfig = r'D:\Impacta\Python for Data Engineer\Aula 2\Arquivos'
configFileName = 'config_analisa_bibtex.yml'

with open(dirConfig + '\\' + configFileName) as f:
    configFile = yml.load(f, Loader=yml.loader.SafeLoader)


In [3]:
# ANALISA BIBTEX
listaArquivos = ( \
	configFile['FILE_ACM'], \
	configFile['FILE_IEE'], \
	configFile['FILE_SD' ]
	)

formatFile = configFile['FORMATO']

dirInput = r'D:\Impacta\Python for Data Engineer\Aula 2\Arquivos\input'
dirOutput = r'D:\Impacta\Python for Data Engineer\Aula 2\Arquivos\output'

mergedFilesCSV = 'ALL_ARTICLES.csv'
mergedFilesJSON = 'ALL_ARTICLES.json'
mergedFilesYAML = 'ALL_ARTICLES.yml'

def bibtexToDict(arqv):
	arquivo = arqv
	with open(dirInput + '\\' + arquivo, encoding='utf8') as f:
		bib_database = bp.load(f)		
	return bib_database.entries_dict.values() # retorna lista de dicionarios, verificar extend
	
# UNION DATAFRAMES AND YML
listaDf = []
listaYml =[]
for arqv in listaArquivos:
	sourceArticles = bibtexToDict(arqv)
	listaDf.append(pd.DataFrame(sourceArticles))
	listaYml.append(yml.dump(list(sourceArticles)))

unionDf = pd.concat(listaDf)
unionYml = ''.join(listaYml)

In [4]:
# EXPORT
if formatFile == 'CSV':
	unionDf.to_csv(dirOutput + '\\' + mergedFilesCSV, index=False, encoding='utf-8')
elif formatFile == 'JSON':
	unionDf.to_json(dirOutput + '\\' + mergedFilesJSON, orient = 'records')
elif formatFile == 'YAML':
	with open(dirOutput + '\\' + mergedFilesYAML, mode='w',encoding='utf8') as f:
		f.write(unionYml)
else:
	print('Formato não disponivel.')


# Article Impact

In [5]:
import pandas as pd
import numpy as np


dir_iput_aula3 = r'D:\Impacta\Python for Data Engineer\Aula 3\Input'
dir_output_aula3 = r'D:\Impacta\Python for Data Engineer\Aula 3\Output'

file_scimagojr = 'scimagojr 2020.csv'
file_jcs = 'jcs_2020.csv'
file_article_impact = 'ALL_Article_Impact.csv'

df_scimagojr = pd.read_csv(dir_iput_aula3 + '\\' + file_scimagojr, delimiter=';', quotechar='"', header=0)
df_jcs = pd.read_csv(dir_iput_aula3 + '\\' + file_jcs, delimiter=';', quotechar='"', header=0)

# TRATAMENTO
# Remove colunas vazias no arquivo
df_scimagojr.drop(df_scimagojr.columns[df_scimagojr.columns.str.contains('Unnamed:') == True], axis=1, inplace=True)
df_jcs.drop(df_jcs.columns[df_jcs.columns.str.contains('Unnamed:') == True], axis=1, inplace=True)


df_jcs.rename(columns={'Full Journal Title':'Title'}, inplace=True)
df_jcs.rename(columns={'Journal Impact Factor':'SJR'}, inplace=True)
df_scimagojr.rename(columns={'Total Cites (3years)':'Total Cites'}, inplace=True)

df_scimagojr['Title'] = df_scimagojr['Title'].str.upper()
df_jcs['Title'] = df_jcs['Title'].str.upper()

# TODAS AS LINHAS DO df_jcs QUE NAO EXISTEM NO df_scimagojr
df_jcs_left_df_scimagojr = df_jcs.merge(df_scimagojr, how='left', on='Title', suffixes=['_jcs','_scimagojr'])
df_jcs_exclusive = df_jcs_left_df_scimagojr[df_jcs_left_df_scimagojr['Rank_scimagojr'].isnull()] 
df_jcs_exclusive.filter(df_jcs.columns)

# UNION
df_scimagojr_jcs = pd.concat([df_scimagojr, df_jcs_exclusive.filter(df_jcs.columns)])

# TRATAMENTO UNION
dict_treat_data = {'': np.nan, None: np.nan, 'Not Available': np.nan}
df_scimagojr_jcs.replace(to_replace=dict_treat_data, inplace=True)
df_scimagojr_jcs['Rank'] = np.arange(0,len(df_scimagojr_jcs))
df_scimagojr_jcs.drop_duplicates()

# EXPORT
df_scimagojr_jcs.to_csv(dir_output_aula3 + '\\' + file_article_impact, index=False, encoding='utf-8')
# df_jcs_exclusive



c:\Users\Saulo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Article Impact e Bibtex

In [6]:

df_bibtex = unionDf
df_bibtex.rename(columns ={'title':'Title'}, inplace=True)
df_bibtex['Title'] = df_bibtex['Title'].str.upper()
dict_treat_data = {'': np.nan, None: np.nan, 'Not Available': np.nan}
df_bibtex.replace(to_replace=dict_treat_data, inplace=True)


In [7]:
df_bibtex_impact = df_bibtex.merge(df_scimagojr_jcs, how='outer', on='Title', suffixes=['_1','_2'])
df_bibtex_impact[df_bibtex_impact['SJR'].notnull() == True]

,series,location,keywords,numpages,pages,booktitle,abstract,doi,url,address,...,Total Cites,Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,Country,Region,Publisher,Coverage,Categories,Eigenfactor Score
225,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15499.0,80.0,"126,34","73,45",United States,Northern America,Wiley-Blackwell,1950-2020,Hematology (Q1); Oncology (Q1),NaN
226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,492.0,9.0,"50,00","129,20",United States,Northern America,Centers for Disease Control and Prevention (CDC),1990-2020,Epidemiology (Q1); Health Information Manageme...,NaN
227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10844.0,167.0,"32,83","73,38",United Kingdom,Western Europe,Nature Publishing Group,2000-2020,Cell Biology (Q1); Molecular Biology (Q1),NaN
228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1945.0,109.0,"16,00","68,33",United Kingdom,Western Europe,Oxford University Press,1886-2020,Economics and Econometrics (Q1),NaN
229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11188.0,138.0,"32,15","115,57",United Kingdom,Western Europe,Nature Publishing Group,2016-2020,"Biomaterials (Q1); Electronic, Optical and Mag...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,45.0,"0,00","0,00",Germany,Western Europe,Harrassowitz Verlag,"1975, 1985, 2002-2009, 2011-2018",Cultural Studies (Q4); History (Q4),NaN
32825,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,55.0,"0,07","56,75",Germany,Western Europe,Walter de Gruyter GmbH,"1861, 1863-1864, 1866-1870, 1872-1873, 1876, 1...",History (Q4); Law (Q4),NaN
32826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,68.0,"0,11","0,00",Germany,Western Europe,Deutscher Kunstverlag GmbH,"1975, 1987, 2001-2019",Visual Arts and Performing Arts (Q4),NaN
32827,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,56.0,"0,03","21,50",Netherlands,Western Europe,Brill Academic Publishers,"1929, 1948-1949, 1951-2000, 2006-2020",History (Q4); Philosophy (Q4); Religious Studi...,NaN
